## CSV TO MONGODB CONVERTER - PYTHON

#### Usage: This script is to help upload csv files to mongodb

#### Maintainer: (Benji Charles)

#### Date: 2019-05-01

#### Version: 1.0

#### Requirements: Python 3.6, pymongo, pandas, python-dotenv

#### How to use this script

1. Create a .env file in the same directory as this script and add the following variables
2. Set the `MONGO_CONNECTION_STRING` in the .env file
3. Populate the `UPLOAD_INFO` list with the following information
    1. The name of the csv file
    2. The name of the collection in mongodb
    3. The name of the database in mongodb



In [12]:
# Import libraries
import pandas
import pymongo
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.




True

In [13]:
# get variables from the user

MONGO_CONNECTION_STRING = os.getenv('MONGO_CONNECTION_STRING')
UPLOAD_INFO = [
    {
    "file": "../data/term2224-fl-senators.csv",
    "database": "FL_125th-Regular-Session",
    "collection": "senators"
    },
    {
    "file": "../data/term2224-fl-representatives.csv",
    "database": "FL_125th-Regular-Session",
    "collection": "representatives"
    }
]

print()


In [14]:
# connect to MongoDB
def get_mongodb_client(db_name,collection_name):
    client = pymongo.MongoClient(MONGO_CONNECTION_STRING)
    db = client.get_database(db_name)
    return db.get_collection(collection_name)

# check if data exists in MongoDB
def check_if_data_exists(database, data):
    returned_data = database.find_one(data)
    # print(returned_data)
    if database.find_one(data):
        
        return True
    return False

# loop through the files and upload the data
for database_info in UPLOAD_INFO:
    # read in the file
    df = pandas.read_csv(database_info["file"])
    
    # if the first column name contains "Unnamed" then remove it
    if "Unnamed" in df.columns.values.tolist()[0]:
        df = df.drop(columns=df.columns.values.tolist()[0])
    
    # normalize the data
    df.reset_index(drop=True, inplace=True)
    
    # turn dataframe into a list of dictionaries
    data_dict = df.to_dict(orient="records")
            
    # connect to MongoDB
    database = get_mongodb_client(database_info["database"], database_info["collection"])
    
    # if the data is not in the database then add to the to_upload_list list
    to_upload_list = []
    for data in data_dict:
        if not check_if_data_exists(database, data):
            to_upload_list.append(data)
                
    # insert the data into MongoDB
    if len(to_upload_list) > 0:
        database.insert_many(to_upload_list)
        
    print(f'{len(to_upload_list)} new records uploaded to {database_info["database"]}.{database_info["collection"]}')
        


0 new records uploaded to FL_125th-Regular-Session.senators
0 new records uploaded to FL_125th-Regular-Session.representatives
